In [13]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor


In [23]:
df = pd.read_csv("C:/Users/harish/Desktop/train.csv",index_col='Id')
Y_train=df['SalePrice']
df=df.iloc[:,:-1]


In [24]:

df['LotFrontage'].fillna(0,inplace=True)
df['Alley'].fillna('NAA',inplace=True)
df['MasVnrType'].fillna('None',inplace=True)
df.update(df[['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']].fillna('NB'))
df['MasVnrArea'].fillna(0,inplace=True)
df['Electrical'].fillna(df['Electrical'].mode()[0],inplace=True)
df['FireplaceQu'].fillna('NFP',inplace=True)
df.update(df[['GarageType','GarageYrBlt','GarageFinish','GarageQual','GarageCond']].fillna('NG'))
df['PoolQC'].fillna('NP',inplace=True)
df['Fence'].fillna('NF',inplace=True)

df.fillna(method='ffill',inplace=True)





In [25]:

df=pd.get_dummies(df,columns=['MSZoning','Neighborhood','Condition1','Condition2','BldgType',
                              'MasVnrType','Exterior2nd','Exterior1st','RoofStyle','Heating',
                              'CentralAir','RoofMatl'],drop_first=True)


df['SaleType']=LabelEncoder().fit_transform(df['SaleType'])
df['SaleCondition']=LabelEncoder().fit_transform(df['SaleCondition'])


In [26]:
replacements = {
   'Street': {'Pave': 1,'Grvl': 0},
    'Alley': {'NAA': 0,'Grvl': 1,'Pave':2},
    'LotShape': {'IR3': 0,'IR2': 1,'IR1':2,'Reg':3},
    'LandContour': {'Low': 0,'HLS': 1,'Bnk':2,'Lvl':3},
    'LotConfig': {'Inside': 0,'Corner': 1,'CulDSac':2,'FR2':3,'FR3':4},
    'LandSlope': {'Sev': 0,'Mod': 1,'Gtl':2},
    'HouseStyle': {'1Story': 1,'1.5Fin':1.5,'1.5Unf':1.25,'2Story':2,'2.5Fin':2.5,'2.5Unf':2.25,'SFoyer':3,'SLvl':3.5},
    'ExterQual': {'Ex': 4,'Gd':3,'TA':2,'Fa':1,'Po':0},
    'ExterCond': {'Ex': 4,'Gd':3,'TA':2,'Fa':1,'Po':0},
    'Foundation': {'PConc':5,'CBlock':4,'Stone':3,'Slab':2,'BrkTil':1,'Wood':0},
    'BsmtQual': {'Ex':105,'Gd':95,'TA':85,'Fa':75,'Po':42.5,'NB':0},
    'BsmtCond': {'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NB':0},
    'BsmtExposure':{'Gd': 4,'Av':3,'Mn':2,'No':1,'NB':0},
    'BsmtFinType1':{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NB':0},
    'BsmtFinType2':{'GLQ':6,'ALQ':5,'BLQ':4,'Rec':3,'LwQ':2,'Unf':1,'NB':0},
    'HeatingQC':{'Ex': 4,'Gd':3,'TA':2,'Fa':1,'Po':0},
    'Electrical':{'SBrkr': 4,'FuseA':3,'FuseF':2,'FuseP':1,'Mix':2.5},
    'KitchenQual':{'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1},
    'Functional':{'Typ': 7,'Min1':6,'Min2':5,'Mod':4,'Maj1':3,'Maj2':2,'Sev':1,'Sal':0},
    'FireplaceQu':{'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NFP':0},
    'GarageType':{'2Types': 6,'Attchd':5,'Basment':4,'BuiltIn':3,'CarPort':2,'Detchd':1,'NG':0},
    'GarageFinish':{'Fin':3,'RFn':2,'Unf':1,'NG':0}, 
    'GarageQual':{'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NG':0},
    'GarageCond':{'Ex': 5,'Gd':4,'TA':3,'Fa':2,'Po':1,'NG':0},
    'PavedDrive':{'Y': 3,'P':2,'N':1},
    'PoolQC':{'Ex': 4,'Gd':3,'TA':2,'Fa':1,'NP':0},
    'Fence':{'GdPrv': 4,'MnPrv':3,'GdWo':2,'MnWw':1,'NF':0}   
}

df.replace(replacements, inplace=True)


df.loc[df['Utilities'] =='AllPub', 'Utilities'] = 1
df.loc[df['Utilities'] != 1, 'Utilities'] = 0


In [27]:
df['basementinfo']=(df['BsmtFinSF1']*df['BsmtFinType2'])+(df['BsmtFinSF1']*df['BsmtFinType2'])+df['BsmtUnfSF']
df['Totalbath']=df['BsmtFullBath']+df['BsmtHalfBath']+df['FullBath']+df['HalfBath']
df['Kitcheninfo']=df['KitchenQual']*df['KitchenAbvGr']
df['Fireplaceinfo']=df['FireplaceQu']*df['Fireplaces']
df['Garageinfo']=df['GarageType']*df['GarageFinish']*df['GarageQual']*df['GarageCond']*df['GarageCars']*df['GarageArea']
df['Porchtotal']=df['OpenPorchSF']+df['EnclosedPorch']+df['3SsnPorch']+df['ScreenPorch']
df['Poolinfo']=df['PoolQC']*df['PoolArea']


In [28]:
df.drop(['BsmtFinSF1', 'BsmtFinSF2','BsmtFinType1','BsmtFinType2','BsmtUnfSF','TotalBsmtSF',
        '1stFlrSF', '2ndFlrSF','LowQualFinSF','KitchenQual', 'KitchenAbvGr','FireplaceQu', 
         'Fireplaces','GarageYrBlt','GarageType','GarageFinish','GarageQual','GarageCond',
         'GarageCars','GarageArea','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch'
        ,'PoolQC', 'PoolArea','MiscFeature','MoSold','BsmtFullBath', 'BsmtHalfBath','FullBath','HalfBath'], axis=1,inplace=True)

In [29]:
cv=5
cvsetsize=int(df.shape[0]/cv)
cvsetsize


X_train1=df.iloc[cvsetsize: , ]
X_test1=df.iloc[: cvsetsize,]
Y_train1=Y_train.iloc[cvsetsize: , ]
Y_test1=Y_train.iloc[: cvsetsize,]

X_train2=pd.concat([df.iloc[:cvsetsize,],df.iloc[2*cvsetsize:,]])
X_test2=df.iloc[cvsetsize:2*cvsetsize,]
Y_train2=pd.concat([Y_train.iloc[:cvsetsize,],Y_train.iloc[2*cvsetsize:,]])
Y_test2=Y_train.iloc[cvsetsize:2*cvsetsize,]

X_train3=pd.concat([df.iloc[:2*cvsetsize,],df.iloc[3*cvsetsize:,]])
X_test3=df.iloc[2*cvsetsize:3*cvsetsize,]
Y_train3=pd.concat([Y_train.iloc[:2*cvsetsize,],Y_train.iloc[3*cvsetsize:,]])
Y_test3=Y_train.iloc[2*cvsetsize:3*cvsetsize,]

X_train4=pd.concat([df.iloc[:3*cvsetsize,],df.iloc[4*cvsetsize:,]])
X_test4=df.iloc[3*cvsetsize:4*cvsetsize,]
Y_train4=pd.concat([Y_train.iloc[:3*cvsetsize,],Y_train.iloc[4*cvsetsize:,]])
Y_test4=Y_train.iloc[3*cvsetsize:4*cvsetsize,]

X_train5=df.iloc[:4*cvsetsize,]
X_test5=df.iloc[4*cvsetsize:,]
Y_train5=Y_train.iloc[:4*cvsetsize,]
Y_test5=Y_train.iloc[4*cvsetsize:,]


In [78]:

model=RandomForestRegressor(n_estimators=150,random_state=6)
RF1=model.fit(X_train1,Y_train1)
RF2=model.fit(X_train2,Y_train2)
RF3=model.fit(X_train3,Y_train3)
RF4=model.fit(X_train4,Y_train4)
RF5=model.fit(X_train5,Y_train5)


print('RANDOM FOREST')
print(RF5.score(X_train5,Y_train5))
print(RF5.score(X_test5,Y_test5))
print('***********************')
print(RF4.score(X_train4,Y_train4))
print(RF4.score(X_test4,Y_test4))
print('***********************')
print(RF3.score(X_train3,Y_train3))
print(RF3.score(X_test3,Y_test3))
print('***********************')
print(RF2.score(X_train2,Y_train2))
print(RF2.score(X_test2,Y_test2))
print('***********************')
print(RF1.score(X_train1,Y_train1))
print(RF1.score(X_test1,Y_test1))
print('***********************')
print('***********************')

print('avg train')
print((RF5.score(X_train5,Y_train5)+RF4.score(X_train4,Y_train4)+RF3.score(X_train3,Y_train3)+RF2.score(X_train2,Y_train2)
     +RF1.score(X_train1,Y_train1))/5)

print('***********************')
print('avg test')
print((RF5.score(X_test5,Y_test5)+RF4.score(X_test4,Y_test4)+RF3.score(X_test3,Y_test3)+RF2.score(X_test2,Y_test2)
     +RF1.score(X_test1,Y_test1))/5)


RANDOM FOREST
0.9804570971142762
0.8197893727935648
***********************
0.9392144795473517
0.9823595496064571
***********************
0.9358613505554002
0.9791344574440785
***********************
0.9380714911945781
0.9774837404950644
***********************
0.9383163470170632
0.9837773599066896
***********************
***********************
avg train
0.9463841530857339
***********************
avg test
0.9485088960491709


In [126]:
model=GradientBoostingRegressor(n_estimators=400,random_state=4,max_depth=4)
GB1=model.fit(X_train1,Y_train1)
GB2=model.fit(X_train2,Y_train2)
GB3=model.fit(X_train3,Y_train3)
GB4=model.fit(X_train4,Y_train4)
GB5=model.fit(X_train5,Y_train5)

print('avg train')
print((GB5.score(X_train5,Y_train5)+GB4.score(X_train4,Y_train4)+GB3.score(X_train3,Y_train3)+GB2.score(X_train2,Y_train2)
     +GB1.score(X_train1,Y_train1))/5)

print('***********************')
print('avg test')
print((GB5.score(X_test5,Y_test5)+GB4.score(X_test4,Y_test4)+GB3.score(X_test3,Y_test3)+GB2.score(X_test2,Y_test2)
     +GB1.score(X_test1,Y_test1))/5)
print('GRADIENT BOOSTING')
print(GB5.score(X_train5,Y_train5))
print(GB5.score(X_test5,Y_test5))
print('***********************')
print(GB4.score(X_train4,Y_train4))
print(GB4.score(X_test4,Y_test4))
print('***********************')
print(GB3.score(X_train3,Y_train3))
print(GB3.score(X_test3,Y_test3))
print('***********************')
print(GB2.score(X_train2,Y_train2))
print(GB2.score(X_test2,Y_test2))
print('***********************')
print(GB1.score(X_train1,Y_train1))
print(GB1.score(X_test1,Y_test1))
print('***********************')
print('ho gaya')



avg train
0.9653072990848823
***********************
avg test
0.9670924830683523
GRADIENT BOOSTING
0.9976937147600001
0.8450534650618902
***********************
0.9590379523449747
0.9969998103019372
***********************
0.9547327394001556
0.9983428610058708
***********************
0.9566360365790328
0.99787649894041
***********************
0.9584360523402486
0.9971897800316534
***********************
ho gaya


In [ ]:
*****************************

In [ ]:
modelLasso=Lasso(alpha=155)
LA1=modelLasso.fit(X_train1,Y_train1)
LA2=modelLasso.fit(X_train2,Y_train2)
LA3=modelLasso.fit(X_train3,Y_train3)
LA4=modelLasso.fit(X_train4,Y_train4)
LA5=modelLasso.fit(X_train5,Y_train5)

modeldt=DecisionTreeRegressor(max_depth=7)
DT1=modeldt.fit(X_train1,Y_train1)
DT2=modeldt.fit(X_train2,Y_train2)
DT3=modeldt.fit(X_train3,Y_train3)
DT4=modeldt.fit(X_train4,Y_train4)
DT5=modeldt.fit(X_train5,Y_train5)

modelKNN=KNeighborsRegressor(n_neighbors=5,weights='distance')
KNN1=modelKNN.fit(X_train1,Y_train1)
KNN2=modelKNN.fit(X_train2,Y_train2)
KNN3=modelKNN.fit(X_train3,Y_train3)
KNN4=modelKNN.fit(X_train4,Y_train4)
KNN5=modelKNN.fit(X_train5,Y_train5)

print('LASSO')
print(LA5.score(X_train5,Y_train5))
print(LA5.score(X_test5,Y_test5))
print('***********************')
print(LA4.score(X_train4,Y_train4))
print(LA4.score(X_test4,Y_test4))
print('***********************')
print(LA3.score(X_train3,Y_train3))
print(LA3.score(X_test3,Y_test3))
print('***********************')
print(LA2.score(X_train2,Y_train2))
print(LA2.score(X_test2,Y_test2))
print('***********************')
print(LA1.score(X_train1,Y_train1))
print(LA1.score(X_test1,Y_test1))
print('***********************')
print('***********************')
print('DECISION TREE')
print(DT5.score(X_train5,Y_train5))
print(DT5.score(X_test5,Y_test5))
print('***********************')
print(DT4.score(X_train4,Y_train4))
print(DT4.score(X_test4,Y_test4))
print('***********************')
print(DT3.score(X_train3,Y_train3))
print(DT3.score(X_test3,Y_test3))
print('***********************')
print(DT2.score(X_train2,Y_train2))
print(DT2.score(X_test2,Y_test2))
print('***********************')
print(DT1.score(X_train1,Y_train1))
print(DT1.score(X_test1,Y_test1))
print('***********************')
print('***********************')
print('K NEAREST NEIGHBORS')
print(KNN5.score(X_train5,Y_train5))
print(KNN5.score(X_test5,Y_test5))
print('***********************')
print(KNN4.score(X_train4,Y_train4))
print(KNN4.score(X_test4,Y_test4))
print('***********************')
print(KNN3.score(X_train3,Y_train3))
print(KNN3.score(X_test3,Y_test3))
print('***********************')
print(KNN2.score(X_train2,Y_train2))
print(KNN2.score(X_test2,Y_test2))
print('***********************')
print(KNN1.score(X_train1,Y_train1))
print(KNN1.score(X_test1,Y_test1))
print('***********************')
print('***********************')
print('RANDOM FOREST')
print(RF5.score(X_train5,Y_train5))
print(RF5.score(X_test5,Y_test5))
print('***********************')
print(RF4.score(X_train4,Y_train4))
print(RF4.score(X_test4,Y_test4))
print('***********************')
print(RF3.score(X_train3,Y_train3))
print(RF3.score(X_test3,Y_test3))
print('***********************')
print(RF2.score(X_train2,Y_train2))
print(RF2.score(X_test2,Y_test2))
print('***********************')
print(RF1.score(X_train1,Y_train1))
print(RF1.score(X_test1,Y_test1))
print('***********************')
print('***********************')
print('GRADIENT BOOSTING')
print(GB5.score(X_train5,Y_train5))
print(GB5.score(X_test5,Y_test5))
print('***********************')
print(GB4.score(X_train4,Y_train4))
print(GB4.score(X_test4,Y_test4))
print('***********************')
print(GB3.score(X_train3,Y_train3))
print(GB3.score(X_test3,Y_test3))
print('***********************')
print(GB2.score(X_train2,Y_train2))
print(GB2.score(X_test2,Y_test2))
print('***********************')
print(GB1.score(X_train1,Y_train1))
print(GB1.score(X_test1,Y_test1))
print('***********************')


In [76]:
corr=df.corr().abs().unstack().sort_values(ascending=False)
corr

YrSold         YrSold           1.000000
GarageCars     GarageCars       1.000000
HalfBath       HalfBath         1.000000
BedroomAbvGr   BedroomAbvGr     1.000000
KitchenAbvGr   KitchenAbvGr     1.000000
TotRmsAbvGrd   TotRmsAbvGrd     1.000000
Fireplaces     Fireplaces       1.000000
GarageYrBlt    GarageYrBlt      1.000000
GarageArea     GarageArea       1.000000
BsmtHalfBath   BsmtHalfBath     1.000000
WoodDeckSF     WoodDeckSF       1.000000
OpenPorchSF    OpenPorchSF      1.000000
EnclosedPorch  EnclosedPorch    1.000000
3SsnPorch      3SsnPorch        1.000000
ScreenPorch    ScreenPorch      1.000000
PoolArea       PoolArea         1.000000
FullBath       FullBath         1.000000
BsmtFullBath   BsmtFullBath     1.000000
MoSold         MoSold           1.000000
MasVnrArea     MasVnrArea       1.000000
LotFrontage    LotFrontage      1.000000
LotArea        LotArea          1.000000
OverallQual    OverallQual      1.000000
OverallCond    OverallCond      1.000000
YearBuilt      Y

In [28]:
y_predLA1=LA1.predict(X_test1)
y_predLA2=LA2.predict(X_test2)
y_predLA3=LA3.predict(X_test3)
y_predLA4=LA4.predict(X_test4)
y_predLA5=LA5.predict(X_test5)

y_predLA=(y_predLA1+y_predLA2+y_predLA3+y_predLA4+y_predLA5)/5

y_predDT1=DT1.predict(X_test1)
y_predDT2=DT2.predict(X_test2)
y_predDT3=DT3.predict(X_test3)
y_predDT4=DT4.predict(X_test4)
y_predDT5=DT5.predict(X_test5)

y_predDT=(y_predDT1+y_predDT2+y_predDT3+y_predDT4+y_predDT5)/5

y_predSVM1=SVM1.predict(X_test1)
y_predSVM2=SVM2.predict(X_test2)
y_predSVM3=SVM3.predict(X_test3)
y_predSVM4=SVM4.predict(X_test4)
y_predSVM5=SVM5.predict(X_test5)



NameError: name 'y_y_predSVM3' is not defined

In [ ]:
s1=pd.Series(X_test.index.values, name='Id')
s2=pd.Series(y_pred, name='SalePrice')
sub= pd.concat([s1, s2], axis=1)
sub.head()

In [ ]:
import os
os.chdir("C:/Users/harish/Desktop")
sub.to_csv('finaldataprocessing.csv',index=False)